In [1]:
import scanpy as sc
import pandas as pd
import random
import matplotlib as mpl
import numpy as np
import os
from scipy.spatial import distance

import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
from matplotlib.lines import Line2D 

import warnings
warnings.filterwarnings('ignore')

from copy import copy
reds = copy(mpl.cm.Reds)
reds.set_under("lightgray")

project_directory = '/Cranio_Lab/Louk_Seton/mesenchyme_project_2023'
os.chdir(os.path.expanduser("~")+project_directory)

In [2]:
adata = sc.read('anndata_objects/interactive/mesen_all.h5ad')
adata.X = adata.layers['original_counts'].copy()
sc.pp.normalize_total(adata, target_sum=10000)
sc.pp.log1p(adata)

In [3]:
xl_file = pd.ExcelFile('Supplementary Table 4 - Genes used for AUCell analysis.xlsx')

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [4]:
dfs['EFO_0007841_0.1'].index = list(dfs['EFO_0007841_0.1']['targetId'])
del dfs['EFO_0007841_0.1']['Unnamed: 0']

In [5]:
gene_list_org = dfs['EFO_0007841_0.1']

In [6]:
lmd_result = pd.read_csv('r_lmd_result.txt',index_col = 0)
lmd_result.rename({'score':'LMD_score'}, axis=1, inplace=True)
gene_list_all = dfs['EFO_0007841_0.1'].merge(lmd_result, how = 'left',left_index=True, right_index=True)
del gene_list_all['rank']

In [7]:
gene_list_all

,targetId,variantRsId,studyId,score,diseaseFromSourceMappedId,LMD_score
Pik3r1,Pik3r1,rs57106444,GCST011429_25,0.489355,EFO_0007841,4.735931
Hmga2,Hmga2,rs7966340,GCST90026542,0.851618,EFO_0007841,NaN
Hmga2,Hmga2,rs10878346,GCST90044778,0.862497,EFO_0007841,NaN
Zeb2,Zeb2,rs148912137,GCST90007187,0.504181,EFO_0007841,3.009557
Hmga2,Hmga2,rs7966340,GCST90026541,0.850571,EFO_0007841,NaN
...,...,...,...,...,...,...
Alx4,Alx4,rs10838269,GCST90007273,0.663373,EFO_0010949,4.726264
Creb3l2,Creb3l2,rs6948022,GCST90007303,0.846874,EFO_0010949,4.572417
Efemp1,Efemp1,rs1367228,GCST90007273,0.835481,EFO_0010949,4.397324
Plpp3,Plpp3,rs35622381,GCST90007273,0.142420,EFO_0010949,4.854394


In [8]:
sc.tl.rank_genes_groups(adata,groupby = 'annotation_all',
                        method = 'wilcoxon',
                       )

In [9]:
rank_genes_df = sc.get.rank_genes_groups_df(adata, group = None)
rank_genes_df = rank_genes_df[rank_genes_df['names'].isin(list(set(gene_list_all['targetId'])))]
rank_genes_df = rank_genes_df[~rank_genes_df['scores'].between(-4, 4)]
rank_genes_df = rank_genes_df.pivot(index='names', columns='group', values=['logfoldchanges'])
for gene in list(set(gene_list_org['targetId']).difference(list(rank_genes_df.index))):
    rank_genes_df.loc[gene] = pd.Series(dtype='float64')

rank_genes_df.columns = rank_genes_df.columns.droplevel()

expanded_table = {'all_cells':gene_list_all.merge(rank_genes_df, left_index=True, right_index=True)}


expanded_table['all_cells'].insert(loc=5, column='Top Cluster', value=list(expanded_table['all_cells'].iloc[:,6:].idxmax(axis=1,skipna=True)))
expanded_table['all_cells']

,targetId,variantRsId,studyId,score,diseaseFromSourceMappedId,Top Cluster,LMD_score,Neural Crest,MXP Early,MXP Early 2,...,Lip - Dermal,Lip Fibroblasts,Lip-Nostril Fusion,Nostrils - Gata2,Nostrils - Pax7,Nostrils - Tfap2b,Philtrum,Roof - Sim1,Whisker Pad,Periocular Mesenchyme
Pik3r1,Pik3r1,rs57106444,GCST011429_25,0.489355,EFO_0007841,Perichondrium,4.735931,-2.658116,-2.627492,-2.948618,...,-0.931847,NaN,NaN,NaN,NaN,-0.847520,1.100682,0.513304,-0.928966,NaN
Hmga2,Hmga2,rs7966340,GCST90026542,0.851618,EFO_0007841,MXP 3,NaN,-0.782482,-0.841932,NaN,...,1.159314,-0.392486,-1.041206,NaN,-1.512490,-0.580599,-1.828394,0.371909,0.304483,NaN
Hmga2,Hmga2,rs10878346,GCST90044778,0.862497,EFO_0007841,MXP 3,NaN,-0.782482,-0.841932,NaN,...,1.159314,-0.392486,-1.041206,NaN,-1.512490,-0.580599,-1.828394,0.371909,0.304483,NaN
Zeb2,Zeb2,rs148912137,GCST90007187,0.504181,EFO_0007841,Roof - Sim1,3.009557,NaN,-2.322127,-4.138664,...,0.521783,0.339233,0.900199,-0.754061,1.216569,-0.319992,1.190975,2.762205,1.734469,NaN
Hmga2,Hmga2,rs7966340,GCST90026541,0.850571,EFO_0007841,MXP 3,NaN,-0.782482,-0.841932,NaN,...,1.159314,-0.392486,-1.041206,NaN,-1.512490,-0.580599,-1.828394,0.371909,0.304483,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Alx4,Alx4,rs10838269,GCST90007273,0.663373,EFO_0010949,LnP - Casz1,4.726264,-4.504507,-5.177282,-2.515970,...,-0.765742,NaN,0.585752,NaN,NaN,0.946143,1.300044,NaN,0.497647,NaN
Creb3l2,Creb3l2,rs6948022,GCST90007303,0.846874,EFO_0010949,Chondrocytes - Septum/Capsule,4.572417,-3.288305,-3.221023,-2.618737,...,-0.376792,NaN,NaN,-0.652480,NaN,-0.733441,NaN,NaN,-0.729403,NaN
Efemp1,Efemp1,rs1367228,GCST90007273,0.835481,EFO_0010949,Lip Fibroblasts,4.397324,-26.972183,-4.876045,-4.704730,...,1.456605,1.798812,-1.444790,NaN,NaN,NaN,NaN,1.105140,1.255099,0.813366
Plpp3,Plpp3,rs35622381,GCST90007273,0.142420,EFO_0010949,Lip Fibroblasts,4.854394,-4.770107,-3.188590,-1.164912,...,NaN,1.074637,0.313572,-0.698780,0.589165,NaN,0.721816,0.329927,0.113914,NaN


In [10]:
for stage in list(adata.obs['stage'].cat.categories):
    adata_tmp = adata[adata.obs['stage']==stage]
    adata_tmp = adata_tmp[adata_tmp.obs['annotation_all'].isin(list(adata_tmp.obs['annotation_all'].value_counts()[adata_tmp.obs['annotation_all'].value_counts()>20].index))]
    sc.tl.rank_genes_groups(adata_tmp,groupby = 'annotation_all',
                        method = 'wilcoxon',
                       )
    rank_genes_df = sc.get.rank_genes_groups_df(adata_tmp, group = None)
    rank_genes_df = rank_genes_df[rank_genes_df['names'].isin(list(set(gene_list_org['targetId'])))]
    rank_genes_df = rank_genes_df[~rank_genes_df['scores'].between(-4, 4)]
    rank_genes_df = rank_genes_df.pivot(index='names', columns='group', values=['logfoldchanges'])
    for gene in list(set(gene_list_org['targetId']).difference(list(rank_genes_df.index))):
        rank_genes_df.loc[gene] = pd.Series(dtype='float64')
    rank_genes_df.columns = rank_genes_df.columns.droplevel()
    
    lmd_result = pd.read_csv('r_lmd_result_E'+stage+'.txt',index_col = 0)
    lmd_result.rename({'score':'LMD_score'}, axis=1, inplace=True)
    gene_list_stage = gene_list_org.merge(lmd_result, how = 'left',left_index=True, right_index=True)
    del gene_list_stage['rank']
    expanded_table[stage] = gene_list_stage.merge(rank_genes_df, left_index=True, right_index=True)

for stage in list(adata.obs['stage'].cat.categories):
    expanded_table[stage].insert(loc=5, column='Top Cluster', value=list(expanded_table[stage].iloc[:,6:].idxmax(axis=1,skipna=True)))

In [11]:
with pd.ExcelWriter("GWAS/gwas_table_expanded_LMD.xlsx") as writer:
    for df_name, df in expanded_table.items():
        df.to_excel(writer, sheet_name=df_name)

In [12]:
xl_file = pd.ExcelFile('Supplementary Table 4 - Genes used for AUCell analysis.xlsx')

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [13]:
dfs['disgenet'].index = list(dfs['disgenet']['mouse_gene'])
del dfs['disgenet']['mouse_gene']

In [14]:
gene_list_org = dfs['disgenet']

In [15]:
lmd_result = pd.read_csv('r_lmd_result.txt',index_col = 0)
lmd_result.rename({'score':'LMD_score'}, axis=1, inplace=True)
gene_list_all = dfs['disgenet'].merge(lmd_result, how = 'left',left_index=True, right_index=True)
del gene_list_all['rank']

In [16]:
gene_list_all

,gene_symbol,numVariantsAssociatedToGene,disease_name,pmid,geneEnsemblIDs,LMD_score
Irf6,IRF6,153,Cleft upper lip,20436469.0,ENSG00000117595,4.554414
Irf6,IRF6,153,Cleft upper lip,18836445.0,ENSG00000117595,4.554414
Irf6,IRF6,153,Cleft Palate,17041601.0,ENSG00000117595,4.554414
Irf6,IRF6,153,Cleft upper lip,17041601.0,ENSG00000117595,4.554414
Irf6,IRF6,153,Cleft upper lip,NaN,ENSG00000117595,4.554414
...,...,...,...,...,...,...
Trp63,TP63,167,Craniofacial Abnormalities,10227294.0,ENSG00000073282,4.738057
Thrb,THRB,188,Craniofacial Abnormalities,10660344.0,ENSG00000151090,4.226990
Col2a1,COL2A1,378,Craniofacial Abnormalities,9061443.0,ENSG00000139219,2.523620
Dlx1,DLX1,0,Craniofacial Abnormalities,9187081.0,ENSG00000144355,2.972744


In [17]:
sc.tl.rank_genes_groups(adata,groupby = 'annotation_all',
                        method = 'wilcoxon',
                       )

In [18]:
rank_genes_df = sc.get.rank_genes_groups_df(adata, group = None)
rank_genes_df = rank_genes_df[rank_genes_df['names'].isin(list(set(gene_list_all.index)))]
rank_genes_df = rank_genes_df[~rank_genes_df['scores'].between(-4, 4)]
rank_genes_df = rank_genes_df.pivot(index='names', columns='group', values=['logfoldchanges'])
for gene in list(set(gene_list_all.index).difference(list(rank_genes_df.index))):
    rank_genes_df.loc[gene] = pd.Series(dtype='float64')

rank_genes_df.columns = rank_genes_df.columns.droplevel()

expanded_table = {'all_cells':gene_list_all.merge(rank_genes_df, left_index=True, right_index=True)}


expanded_table['all_cells'].insert(loc=5, column='Top Cluster', value=list(expanded_table['all_cells'].iloc[:,6:].idxmax(axis=1,skipna=True)))
expanded_table['all_cells']

,gene_symbol,numVariantsAssociatedToGene,disease_name,pmid,geneEnsemblIDs,Top Cluster,LMD_score,Neural Crest,MXP Early,MXP Early 2,...,Lip - Dermal,Lip Fibroblasts,Lip-Nostril Fusion,Nostrils - Gata2,Nostrils - Pax7,Nostrils - Tfap2b,Philtrum,Roof - Sim1,Whisker Pad,Periocular Mesenchyme
Irf6,IRF6,153,Cleft upper lip,20436469.0,ENSG00000117595,NaN,4.554414,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Irf6,IRF6,153,Cleft upper lip,18836445.0,ENSG00000117595,NaN,4.554414,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Irf6,IRF6,153,Cleft Palate,17041601.0,ENSG00000117595,NaN,4.554414,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Irf6,IRF6,153,Cleft upper lip,17041601.0,ENSG00000117595,NaN,4.554414,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Irf6,IRF6,153,Cleft upper lip,NaN,ENSG00000117595,NaN,4.554414,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Trp63,TP63,167,Craniofacial Abnormalities,10227294.0,ENSG00000073282,NaN,4.738057,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Thrb,THRB,188,Craniofacial Abnormalities,10660344.0,ENSG00000151090,Nasal Cavity - Ventral,4.226990,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.461677,NaN,NaN
Col2a1,COL2A1,378,Craniofacial Abnormalities,9061443.0,ENSG00000139219,Chondrocytes - Septum/Capsule,2.523620,-2.492457,-1.276198,0.057825,...,-1.464558,-1.347652,-1.759115,-2.329834,-2.486367,-2.131100,-1.417586,-1.637389,-1.513544,-1.605205
Dlx1,DLX1,0,Craniofacial Abnormalities,9187081.0,ENSG00000144355,Molar,2.972744,NaN,2.037174,3.222830,...,2.116504,NaN,-3.492601,NaN,NaN,-3.118024,NaN,-3.524400,NaN,NaN


In [19]:
for stage in list(adata.obs['stage'].cat.categories):
    adata_tmp = adata[adata.obs['stage']==stage]
    adata_tmp = adata_tmp[adata_tmp.obs['annotation_all'].isin(list(adata_tmp.obs['annotation_all'].value_counts()[adata_tmp.obs['annotation_all'].value_counts()>20].index))]
    sc.tl.rank_genes_groups(adata_tmp,groupby = 'annotation_all',
                        method = 'wilcoxon',
                       )
    rank_genes_df = sc.get.rank_genes_groups_df(adata_tmp, group = None)
    rank_genes_df = rank_genes_df[rank_genes_df['names'].isin(list(set(gene_list_all.index)))]
    rank_genes_df = rank_genes_df[~rank_genes_df['scores'].between(-4, 4)]
    rank_genes_df = rank_genes_df.pivot(index='names', columns='group', values=['logfoldchanges'])
    for gene in list(set(gene_list_all.index).difference(list(rank_genes_df.index))):
        rank_genes_df.loc[gene] = pd.Series(dtype='float64')
    rank_genes_df.columns = rank_genes_df.columns.droplevel()
    
    lmd_result = pd.read_csv('r_lmd_result_E'+stage+'.txt',index_col = 0)
    lmd_result.rename({'score':'LMD_score'}, axis=1, inplace=True)
    gene_list_stage = gene_list_org.merge(lmd_result, how = 'left',left_index=True, right_index=True)
    del gene_list_stage['rank']
    expanded_table[stage] = gene_list_stage.merge(rank_genes_df, left_index=True, right_index=True)

for stage in list(adata.obs['stage'].cat.categories):
    expanded_table[stage].insert(loc=5, column='Top Cluster', value=list(expanded_table[stage].iloc[:,6:].idxmax(axis=1,skipna=True)))

In [20]:
with pd.ExcelWriter("GWAS/gwas_table_disgenet_expanded_LMD.xlsx") as writer:
    for df_name, df in expanded_table.items():
        df.to_excel(writer, sheet_name=df_name)

In [21]:
from math import ceil
from kneed import KneeLocator

lmd_result = pd.read_csv('r_lmd_result.txt',index_col = 0)
kneedle = KneeLocator(lmd_result['rank'], lmd_result['score'], S=2.5, curve="concave", direction="increasing")
ax = lmd_result.plot.scatter(x = 'rank',y = 'score', s = 1)
plt.axhline(y=kneedle.knee_y, color='r', linestyle='-')
plt.axvline(x=kneedle.knee, color='r', linestyle='-')
plt.title('all cells - knee: '+str(kneedle.knee)+' , '+str(ceil(kneedle.knee_y * 1000) / 1000.0))

plt.savefig('GWAS/lmd_all_cells.svg')
plt.close()

for stage in list(adata.obs['stage'].cat.categories):
    lmd_result = pd.read_csv('r_lmd_result_E'+stage+'.txt',index_col = 0)
    kneedle = KneeLocator(lmd_result['rank'], lmd_result['score'], S=2.5, curve="concave", direction="increasing")
    ax = lmd_result.plot.scatter(x = 'rank',y = 'score', s = 1)
    plt.axhline(y=kneedle.knee_y, color='r', linestyle='-')
    plt.axvline(x=kneedle.knee, color='r', linestyle='-')
    plt.title('E'+stage+' - knee: '+str(kneedle.knee)+' , '+str(ceil(kneedle.knee_y * 1000) / 1000.0))
    plt.savefig('GWAS/lmd_E'+stage+'.svg')
    plt.close()

In [23]:
xl_file = pd.ExcelFile('Supplementary Table 4 - Genes used for AUCell analysis.xlsx')

dfs = {sheet_name: xl_file.parse(sheet_name) 
          for sheet_name in xl_file.sheet_names}

In [26]:
dfs['MGI_prominences'].index = list(dfs['MGI_prominences']['Marker Symbol'])


In [27]:
gene_list_org = dfs['MGI_prominences']

In [28]:
lmd_result = pd.read_csv('r_lmd_result.txt',index_col = 0)
lmd_result.rename({'score':'LMD_score'}, axis=1, inplace=True)
gene_list_all = dfs['MGI_prominences'].merge(lmd_result, how = 'left',left_index=True, right_index=True)
del gene_list_all['rank']

In [29]:
gene_list_all

,MGI Marker Accession ID,Marker Symbol,Mammalian Phenotype ID,LMD_score
Dlx5,MGI:101926,Dlx5,MP:0009902,3.922977
Dlx5,MGI:101926,Dlx5,MP:0030250,3.922977
Aldh1a2,MGI:107928,Aldh1a2,MP:0030249,3.104074
Pax6,MGI:97490,Pax6,MP:0009902,4.854568
Pax6,MGI:97490,Pax6,MP:0009903,4.854568
Hesx1,MGI:96071,Hesx1,MP:0009901,4.167228
Pdgfra,MGI:97530,Pdgfra,MP:0009901,5.172327
Sp8,MGI:2443471,Sp8,MP:0009902,4.822944
Sp8,MGI:2443471,Sp8,MP:0009903,4.822944
Sp8,MGI:2443471,Sp8,MP:0010940,4.822944


In [30]:
sc.tl.rank_genes_groups(adata,groupby = 'annotation_all',
                        method = 'wilcoxon',
                       )

In [31]:
rank_genes_df = sc.get.rank_genes_groups_df(adata, group = None)
rank_genes_df = rank_genes_df[rank_genes_df['names'].isin(list(set(gene_list_all['Marker Symbol'])))]
rank_genes_df = rank_genes_df[~rank_genes_df['scores'].between(-4, 4)]
rank_genes_df = rank_genes_df.pivot(index='names', columns='group', values=['logfoldchanges'])
for gene in list(set(gene_list_org['Marker Symbol']).difference(list(rank_genes_df.index))):
    rank_genes_df.loc[gene] = pd.Series(dtype='float64')

rank_genes_df.columns = rank_genes_df.columns.droplevel()

expanded_table = {'all_cells':gene_list_all.merge(rank_genes_df, left_index=True, right_index=True)}


expanded_table['all_cells'].insert(loc=3, column='Top Cluster', value=list(expanded_table['all_cells'].iloc[:,4:].idxmax(axis=1,skipna=True)))
expanded_table['all_cells']

,MGI Marker Accession ID,Marker Symbol,Mammalian Phenotype ID,Top Cluster,LMD_score,Neural Crest,MXP Early,MXP Early 2,MXP 1,MXP 2,...,Lip - Dermal,Lip Fibroblasts,Lip-Nostril Fusion,Nostrils - Gata2,Nostrils - Pax7,Nostrils - Tfap2b,Philtrum,Roof - Sim1,Whisker Pad,Periocular Mesenchyme
Dlx5,MGI:101926,Dlx5,MP:0009902,Osteoblast,3.922977,NaN,-3.930424,NaN,NaN,NaN,...,-1.219709,NaN,NaN,NaN,NaN,-2.217777,-1.919984,-2.297477,-1.072130,NaN
Dlx5,MGI:101926,Dlx5,MP:0030250,Osteoblast,3.922977,NaN,-3.930424,NaN,NaN,NaN,...,-1.219709,NaN,NaN,NaN,NaN,-2.217777,-1.919984,-2.297477,-1.072130,NaN
Aldh1a2,MGI:107928,Aldh1a2,MP:0030249,MxP Mesenchyme,3.104074,-6.854046,-6.205167,NaN,NaN,NaN,...,-2.609440,NaN,-2.602859,-2.594866,NaN,-3.803557,-2.889247,-2.601304,-3.193179,1.237010
Pax6,MGI:97490,Pax6,MP:0009902,NaN,4.854568,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pax6,MGI:97490,Pax6,MP:0009903,NaN,4.854568,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hesx1,MGI:96071,Hesx1,MP:0009901,NaN,4.167228,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pdgfra,MGI:97530,Pdgfra,MP:0009901,Lip Fibroblasts,5.172327,-3.052208,-2.813557,-2.058467,NaN,NaN,...,0.177607,0.992973,NaN,NaN,NaN,NaN,0.646511,NaN,0.181480,NaN
Sp8,MGI:2443471,Sp8,MP:0009902,NaN,4.822944,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sp8,MGI:2443471,Sp8,MP:0009903,NaN,4.822944,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sp8,MGI:2443471,Sp8,MP:0010940,NaN,4.822944,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
for stage in list(adata.obs['stage'].cat.categories):
    adata_tmp = adata[adata.obs['stage']==stage]
    adata_tmp = adata_tmp[adata_tmp.obs['annotation_all'].isin(list(adata_tmp.obs['annotation_all'].value_counts()[adata_tmp.obs['annotation_all'].value_counts()>20].index))]
    sc.tl.rank_genes_groups(adata_tmp,groupby = 'annotation_all',
                        method = 'wilcoxon',
                       )
    rank_genes_df = sc.get.rank_genes_groups_df(adata_tmp, group = None)
    rank_genes_df = rank_genes_df[rank_genes_df['names'].isin(list(set(gene_list_org['Marker Symbol'])))]
    rank_genes_df = rank_genes_df[~rank_genes_df['scores'].between(-4, 4)]
    rank_genes_df = rank_genes_df.pivot(index='names', columns='group', values=['logfoldchanges'])
    for gene in list(set(gene_list_org['Marker Symbol']).difference(list(rank_genes_df.index))):
        rank_genes_df.loc[gene] = pd.Series(dtype='float64')
    rank_genes_df.columns = rank_genes_df.columns.droplevel()
    
    lmd_result = pd.read_csv('r_lmd_result_E'+stage+'.txt',index_col = 0)
    lmd_result.rename({'score':'LMD_score'}, axis=1, inplace=True)
    gene_list_stage = gene_list_org.merge(lmd_result, how = 'left',left_index=True, right_index=True)
    del gene_list_stage['rank']
    expanded_table[stage] = gene_list_stage.merge(rank_genes_df, left_index=True, right_index=True)

for stage in list(adata.obs['stage'].cat.categories):
    expanded_table[stage].insert(loc=3, column='Top Cluster', value=list(expanded_table[stage].iloc[:,4:].idxmax(axis=1,skipna=True)))

In [33]:
with pd.ExcelWriter("GWAS/gwas_table_prominences_expanded_LMD.xlsx") as writer:
    for df_name, df in expanded_table.items():
        df.to_excel(writer, sheet_name=df_name)